In [ ]:
import keras
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mokshjayanthgr
Your Kaggle Key: ··········


100%|██████████| 612M/612M [00:04<00:00, 148MB/s]


In [ ]:
import os
import random

In [ ]:
import shutil

In [ ]:
start_path = "/content/gtsrb-german-traffic-sign/Train/"
end_path = "/content/sample_data/Test/"
os.mkdir("/content/sample_data/Test")

In [ ]:
for i in range(40):
  start = start_path+str(i)+'/'
  end = end_path+str(i)
  os.mkdir(end)
  images = os.listdir(start)
  for i in range(10):
    img = random.choice(images)
    shutil.move(start+img,end+'/'+img)
    images.remove(img)


In [ ]:
for i in range(40):
  print(len(os.listdir(start_path+str(i))))

In [ ]:
#for testing let's get 10 images from each 
path_test = "/content/sample_data"

In [ ]:

# from keras.utils import to_categorical
import tensorflow as tf


In [ ]:
%cd /content/drive/MyDrive/project-4

In [ ]:
os.getcwd()

In [ ]:
data = []
labels = []
classes = 40
curr_path = "/content/gtsrb-german-traffic-sign/Train"
#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(curr_path,str(i))
    images = os.listdir(path)

    for a in images:
          image = Image.open(path + '/'+ a)
          image = image.resize((30,30))
          image = np.array(image)
          data.append(image)
          labels.append(i)
#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)
#Splitting training and testing dataset

(37969, 30, 30, 3) (37969,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(30375, 30, 30, 3) (7594, 30, 30, 3) (30375,) (7594,)


In [ ]:
#Converting the labels into one hot encoding
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:

#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(40, activation='softmax'))

#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")

Epoch 1/15
950/950 [==============================] - 140s 146ms/step - loss: 1.9976 - accuracy: 0.4961 - val_loss: 0.5751 - val_accuracy: 0.8559
Epoch 2/15
712/950 [=====================>........] - ETA: 30s - loss: 0.7951 - accuracy: 0.7720

In [ ]:
def load_and_prep_image(filename, img_shape=30):
  img = tf.io.read_file(filename)

  img = tf.image.decode_image(img, channels=3)

  img = tf.image.resize(img, size = [img_shape, img_shape])
  img/=255.
  return img

In [ ]:
from matplotlib import pyplot as plt
def pred_and_plot(model, filename, class_names):
  img = load_and_prep_image(filename)

  pred = model.predict(tf.expand_dims(img, axis=0))

  # predicted class
  pred_class = class_names[pred.argmax()] # if more than one output, take the max

  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
# image = Image.open("/content/00014_00000_00029.png")
# image = image.resize((30,30))
# image = np.array(image)
# image = np.expand_dims(image,axis=0)
# pred = model.predict(image)
# np.argmax(pred)

In [ ]:
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('my_model.h5')

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
l = plot_model(model)

In [ ]:
l

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('my_model.h5')

In [ ]:
targets = [i for i in range(40)]
targets[0]  = "Speed limit 20 kilometers per hour"
targets[1]  = "Speed limit 30 kilometers per hour"
targets[2]  = "Speed limit 50 kilometers per hour"
targets[3]  = "Speed limit 60 kilometers per hour"
targets[4]  = "Speed limit 70 kilometers per hour"
targets[5]  = "Speed limit 80 kilometers per hour"
targets[6]  = "open road Speed limit of 80 kilometers per hour"
targets[7]  = "Speed limit 100 kilometers per hour"
targets[8]  = "Speed limit 120 kilometers per hour"
targets[9]  = "No overtaking"
targets[10] = "No truck overtaking"
targets[11] = "right of way"
targets[12] = "Closed quadrilateralRoad"
targets[13] = "Closed triangular Road"
targets[14] = "Stop Sign"
targets[15] = "No vehicles allowed"
targets[16] = "Truck parking area"
targets[17] = "No Entry"
targets[18] = "Warning"
targets[19] = "Left curve ahead"
targets[20] = "Right curve ahead"
targets[21] = "Sharp curve ahead"
targets[22] = "Speed bump ahead"
targets[23] = "Slippary Road"
targets[24] = "Narrow road ahead"
targets[25] = "Construction work ahead"
targets[26] = "Traffic sign ahead"
targets[27] = "Crosswalk"
targets[28] = "Children playground ahead"
targets[29] = "Only bicycles allowed"
targets[30] = "Snowy road"
targets[31] = "Animal crossings"
targets[32] = "Derestriction"
targets[33] = "Only right Turn"
targets[34] = "Only Left Turn"
targets[35] = "Keep Straight"
targets[36] = "Keep Straight or Turn Right"
targets[37] = "Keep Straight or Turn Left"
targets[38] = "Sharp turn right back"
targets[39] = "Sharp turn left back"

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
!pip install -q gradio

In [ ]:
!pip install twilio

In [ ]:
import gradio as gr
from twilio.rest import Client

def traffic_app(TrafficSignBoard, PhoneNumber):
    image = TrafficSignBoard
    image = np.expand_dims(image,axis=0)
    pred = model.predict(image)
    pred = np.argmax(pred)

    account_sid = "ACb103647d6f07ddc78a87b76a00b1e779"
    auth_token = "414988b2430c6f3d913878307f97f688"
    client = Client(account_sid, auth_token)

    message = client.messages \
      .create(
          body= "\nSign Board Ahead!\n"+ targets[pred] + ". Drive Safe!",
          from_='+13853868323',
          to=PhoneNumber
      )

    return targets[pred]

demo = gr.Interface(traffic_app, [gr.Image(shape=(30, 30)), "text"], "text",
                    title="Traffic Sign Detection App",
                    description="Follow Traffic Rules. Drive Safe!")

demo.launch(share=True)